## Imports

In [43]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [44]:
%matplotlib qt

## Camera Calibration

In [45]:
def get_camera_calibration(path='camera_cal/calibration*.jpg', show=False, save_path=None):
    '''
    Compute and return camera matrix (mtx) and distortion coefficients (dist)
        based on checkerboard images stored in location indicated by `path`.
        Optional argument `show` displays checherboard images with found corners
        highlated for 0.5s. Optional argument `save_path` saves these images for
        off-line viewing
    '''
    
    # prepare object points
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob(path)


    # Step through the list and search for chessboard corners
    for i, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)


        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
           
            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (9,6), corners, ret)

            if save_path is not None:
                cv2.imwrite(save_path + 'chess_corners{}.jpg'.format(i),img)

            if show is True:
                cv2.imshow('img',img)
                cv2.waitKey(500)
    
    if show is True:
        cv2.destroyAllWindows()
        
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
        
    return mtx, dist

### Compute camera matrix and distortion coefficients

In [46]:
mtx, dist = get_camera_calibration()

## Distortion Correction

In [47]:
def undistort_img(img, mtx, dist):
    ''' 
    Return undistorted image based on camera matrix `mtx` and distortion coefficients `dist`
    '''
    return cv2.undistort(img, mtx, dist, None, mtx)

## Color and Gradient Thresholding

In [48]:
def hls_select(img, thresh=(0, 255)):
    '''
    Return a binary image where pixels which are within the threshold `thresh` of the 
    S-channel (Saturation) in the HSL colorspace (Hue, Saturation, Lightness) are set
    to 255. The rest of the pixels are zero.
    '''
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 255

    return binary_output

In [49]:

def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    '''
    Return a binary image where pixels which are within the threshold `thresh_min`
    and `threshold_max` of the sobel gradient along `orient` direction are set to 
    255. The rest of the pixels are zero.
        
    '''
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)

    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 255

    # Return the result
    return binary_output

In [50]:
def region_of_interest(img):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    
    h,w = img.shape[0], img.shape[1]
        
    vertices = np.array([[[int(0.15*w),int(.95*h)], # bot left
                          [int(.45*w),int(.58*h)],  # top left
                          [int(.55*w),int(.58*h)],  # top right
                          [int(.9*w),int(.95*h)]]],# bot right
                            dtype=np.int32 )
    
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [51]:
def color_and_grandient_threshold(img):
    '''
    Perform color and X-gradient thresholding of the input `img` and return a binary
    image with the pixels that satisfy either color or grandient threshold set to 255.
    '''
    s_binary = hls_select(img, thresh=(170,255))
    sxbinary = abs_sobel_thresh(img, orient='x', thresh_min=20, thresh_max=150)
    combined_binary = np.zeros_like(s_binary)
    combined_binary[(s_binary == 255) | (sxbinary == 255)] = 255
    out = region_of_interest(combined_binary)
    
    return out 

## Perspective Transform

In [52]:
def get_perspective_transform(img_shape):
    '''
    Compute and return prespective transform matrix `M` based on 
    coordinates of quadrangle vertices in the source image and corresponding 
    quadrangle vertices in the destination image. Also compute and return the inverse
    transform matrix `Minv`.
    '''
    
    h,w = img_shape[0], img_shape[1]
    
    top_most = 460
    offset = 50
    
    src = np.float32(
        [[579, top_most],
         [203, h],
         [1127, h],
         [704, top_most]])

    dst = np.float32(
        [[320+offset, 0],
         [320+offset, h],
         [960-offset, h],
         [960-offset, 0]])
    
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    return M, Minv

### Compute perspective transform and its inverse

In [53]:
M, Minv = get_perspective_transform((720,1280))

In [54]:
def warp(img, M):
    '''
    Apply perspective transform to image `img` and return the transformed or warped image
    '''
    img_size = (img.shape[1], img.shape[0])
    return cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

## Line Identification and Curve Fitting

In [55]:
def find_baseline_x(img, lane='left'):
    '''
    Find the location of the 'hot' pixels at the bottom of the image searching either
    left or right half based on parameter `lane`. Return x location of the line.
    '''
    
    # histogram for lower half
    histogram = np.sum(img[np.int(img.shape[0]/2):,:], axis=0)
    
    # find midpoint along x
    midpoint = np.int(histogram.shape[0]/2)
    
    if lane is 'left':
        x_base = np.argmax(histogram[:midpoint])
    else:
        x_base = np.argmax(histogram[midpoint:]) + midpoint
    
    return x_base

In [56]:
def sliding_window_search(img, x_base, debug=False, out_img=None):    
    '''
    Perform sliding window search based on the previously identified starting
    location at the bottom of the image `x_base`. The search is performed in 
    each of several bands along the y axis starting at the bottom in the small
    x axis neighborhood of the previously identified x location in the lower window
    or the x_base in the case of the bottom most window. Once the pixels in all
    windows are identified they are fitted with a 2nd order polynomial whose
    coefficients are returned in `fit`. The radius of curvature (ROC) and the 
    x location of the line are computed using the helper function `roc_and_loc()`
    and are also returned. Debugging information is optionally recorded on the 
    image `out_img` when `debug` flag is set.
    '''
    
    # set parameters
    # number of sliding windows (height dir)
    nwindows = 9
    
    # width of the sliding search window is +/- margin
    margin = 100
    # recenter next window if at least this many points were found
    minpix = 50
    
    #if debug is True:
    #    out_img = np.dstack((img, img, img))*255
    
    # Set height of windows
    window_height = np.int(img.shape[0]/nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated for each window
    x_current = x_base
    
    # Create empty lists to receive left and right lane pixel indices
    lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y
        win_y_low = img.shape[0] - (window+1)*window_height
        win_y_high = img.shape[0] - window*window_height
        
        win_x_low = x_current - margin
        win_x_high = x_current + margin
        
        if debug is True and out_img is not None:
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_x_low,win_y_low),(win_x_high,win_y_high),(0,255,0), 2)
            
        
        # Identify the nonzero pixels in x and y within the window
        good_inds = ((nonzeroy >= win_y_low) &
                     (nonzeroy < win_y_high) & 
                     (nonzerox >= win_x_low) & 
                     (nonzerox < win_x_high)).nonzero()[0]
        
        # Append these indices to the lists
        lane_inds.append(good_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_inds) > minpix:
            x_current = np.int(np.mean(nonzerox[good_inds]))


    # Concatenate the arrays of indices
    lane_inds = np.concatenate(lane_inds)

    # Extract left and right line pixel positions
    x = nonzerox[lane_inds]
    y = nonzeroy[lane_inds] 
    
    if debug is True and out_img is not None:
        out_img[nonzeroy[lane_inds], nonzerox[lane_inds]] = [255, 0, 0]


    # Fit a second order polynomial
    fit = np.polyfit(y, x, 2)
    
    # find ROC and lane location at bottom of image
    r, loc = roc_and_loc(x, y)
    
    #if debug is True:
    #    plt.figure(figsize=(8,10), tight_layout=True)
    #    plt.imshow(out_img)

    return fit, r, loc, out_img

In [57]:
def local_fit_search(img, fit, debug=False, out_img=None):
    '''
    Perform line indentification search along a previously found polynomial `fit`. Return
    updated fit as well a radius of curvature and location in the lane.
    '''
    
    # search based on fit from pervious images
    
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    margin = 100
    
    lane_inds = ((nonzerox > (fit[0]*(nonzeroy**2) + fit[1]*nonzeroy + fit[2] - margin)) &
                 (nonzerox < (fit[0]*(nonzeroy**2) + fit[1]*nonzeroy + fit[2] + margin))) 

    # Extract line pixel positions
    x = nonzerox[lane_inds]
    y = nonzeroy[lane_inds] 
    
    # Fit a second order polynomial
    fit = np.polyfit(y, x, 2)
    
    if debug is True and out_img is not None:
        out_img[nonzeroy[lane_inds], nonzerox[lane_inds]] = [255, 0, 0]
        
        
    # find ROC and lane location at bottom of image
    r, loc = roc_and_loc(x, y)

    return fit, r, loc, out_img

## Radius of Curvature and Line Location Estimation

In [58]:
def roc_and_loc(x, y):
    '''
    Compute and return radius of curvature and line location along the x dir
    in units of meters
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    y_eval_m = 720*ym_per_pix
    x = x*xm_per_pix
    y = y*ym_per_pix
    
    fit = np.polyfit(y, x, 2)
    
    r = ((1 + (2.*fit[0]*y_eval_m + fit[1])**2)**1.5) / np.absolute(2*fit[0])
    
    # evaluate fit at the bottom of image
    loc = fit[0]*y_eval_m**2 + fit[1]*y_eval_m + fit[2]
    
    center_of_image_m = (1280. / 2.) * xm_per_pix
    
    loc -= center_of_image_m
    
    return r, loc

## Video Frame Qualification and Filtering

In [59]:
def fit_param_filter(line, alpha):
    '''
    First order filter
    '''
    line.best_fit = alpha * line.current_fit + (1. - alpha) * line.best_fit

In [60]:
'''
Class to keep track of line fit parameters from frame to frame
'''
class Line():
    def __init__(self, name):
        # name of line 'left' or 'right'
        self.name = name
        
        # car base
        self.L = 2.7 # Lincoln MKZ
        
        # maximum steering magnitude
        self.steering_tol = .01 # rad
        
        # maximumm steerign change from previous frame
        self.steering_change_tol = .0016 # rad
        
        # maximum center change from previous frame
        self.centering_tol = 0.2 # m
        
        # was the line detected in the last iteration?
        self.detected = False 
        
        # disqualifed frame
        self.bad_fit = True
        
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])] 
        
        #radius of curvature of the line in some units
        self.radius_of_curvature = None
        
        # redius of curvature of the previous frame
        self.radius_of_curvature_prev = None
        
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        
        # line location of the previous frame
        self.line_base_pos_prev = None
        
        # counter to keep track of disqualifed frames
        self.bad_fit_cnt = 0

In [61]:
def line_search(img, line):
    '''
    Search for lines in image using either sliding window search or local fit search
    depending on weather previous fit is available 
    '''
    if line.detected is False:
        x_base = find_baseline_x(img, line.name)
        line.current_fit, line.radius_of_curvature, line.line_base_pos, _ = \
            sliding_window_search(img, x_base)
        
            
        line.radius_of_curvature_prev = line.radius_of_curvature
        line.line_base_pos_prev = line.line_base_pos
        line.best_fit = line.current_fit
        
    else:
        line.current_fit, line.radius_of_curvature, line.line_base_pos, _ = \
            local_fit_search(img, line.best_fit)   

In [62]:
def line_qualify(line):
    '''
    Check if line meets requirements based on steering angle change from previous frame,
    absolute steering angle, and line location based change from previous frame.
    '''
    line.bad_fit = False
    
    # convert ROC to steering angle
    steering = line.L / line.radius_of_curvature
    steering_prev = line.L / line.radius_of_curvature_prev
    
    if np.fabs(steering - steering_prev) > line.steering_change_tol:
        line.bad_fit = True
        #print(line.name+'steering_change_tol')
        
    if np.fabs(steering) > line.steering_tol:
        line.bad_fit = True
        #print(line.name+'steering_tol')
        
    if np.fabs(line.line_base_pos - line.line_base_pos_prev) > line.centering_tol:
        line.bad_fit = True
        #print(line.name+'centering_tol')
        
    if line.bad_fit is not True:
        # filter
        fit_param_filter(line, 0.20)
        
        # record previous values
        line.radius_of_curvature_prev = line.radius_of_curvature
        line_base_pos_prev = line.line_base_pos
        line.detected = True
        
        if line.bad_fit_cnt > 0:
            line.bad_fit_cnt -= 1
        
    else:
        line.bad_fit_cnt += 1
        
    if line.bad_fit_cnt > 20:
        line.detected = False
        

In [63]:
def process_image(img, line_l, line_r):
    '''
    Implement the processing pipeline, process a raw input video frame image and return
    an image with the line markings and ROC and LOC data displayed.
    '''
    
    # distortion correction
    img = undistort_img(img, mtx, dist)
    
    # color and gradient thresholding
    img_th = color_and_grandient_threshold(img)
    
    # perspective transform
    img_th = warp(img_th, M)
    
    # line identification
    line_search(img_th, line_l)
    line_search(img_th, line_r)
    
    # line qualification
    line_qualify(line_l)
    line_qualify(line_r)
    
    #loc_error = loc_left + loc_right
    
    # line drawing and perspetive inverse transform
    out = draw_overlay(img, line_l.best_fit, line_r.best_fit, Minv)
    
    return out

## Visual Display of the Output

In [64]:
def draw_overlay(img, fit_l, fit_r, Minv):
    '''
    Display line markings and radius of curvature and lane centering information
    on image `img` using fit data `fit_l` and `fit_r` for left and right lines 
    respectively. Unwarp the image back to camera space using `Minv` perspective
    transformation matrix.
    '''
    
    # Create an image to draw the lines on
    color_warp = np.zeros_like(img).astype(np.uint8)
    #color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    ploty = np.linspace(0, 719, num=720)# to cover same y-range as image
    left_fitx = fit_l[0]*ploty**2 + fit_l[1]*ploty + fit_l[2]
    right_fitx = fit_r[0]*ploty**2 + fit_r[1]*ploty + fit_r[2]
    
    # recompute ROC and LOC based on filtered fit
    r_left, loc_l = roc_and_loc(left_fitx, ploty)
    r_right, loc_r = roc_and_loc(right_fitx, ploty)
    loc_error = loc_l + loc_r

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
    
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(result, '{:s}: {:>+5.2f} m'.format('lane centering', loc_error), (450,700), font, 1,(255,255,255),2)
    cv2.putText(result, '{:s}: {:+.2f} m'.format('ROC', r_left), (50,700), font, 1,(255,255,255),2)
    cv2.putText(result, '{:s}: {:+.2f} m'.format('ROC', r_right), (970,700), font, 1,(255,255,255),2)
    
    return result

## Results

In [65]:
def make_image_processor():
    '''
    Create an video image processor that can be passed to moviepy 
    '''
    line_l = Line('left')
    line_r = Line('right')
    
    def image_processor(img):
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        out = process_image(img, line_l, line_r)
        return cv2.cvtColor(out, cv2.COLOR_BGR2RGB)

    return image_processor

In [66]:
proc = make_image_processor()

In [67]:
output_video = 'output_video_try.mp4'
clip = VideoFileClip("project_video.mp4")
out_clip = clip.fl_image(proc)
out_clip.write_videofile(output_video, audio=False)

[MoviePy] >>>> Building video output_video_try.mp4
[MoviePy] Writing video output_video_try.mp4


100%|█████████▉| 1260/1261 [02:57<00:00,  6.93it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_video_try.mp4 

